In [1]:
##########################################################################################
#                                                                                        #
#  888888b.  8888888 .d88888b.   .d8888b.     .d8888b.   .d8888b.      d8888      d8888  #
#  888  "88b   888  d88P" "Y88b d88P  Y88b   d88P  Y88b d88P  Y88b    d8P888     d8P888  #
#  888  .88P   888  888     888 Y88b.        888        888          d8P 888    d8P 888  #
#  8888888K.   888  888     888  "Y888b.     888d888b.  888d888b.   d8P  888   d8P  888  #
#  888  "Y88b  888  888     888     "Y88b.   888P "Y88b 888P "Y88b d88   888  d88   888  #
#  888    888  888  888     888       "888   888    888 888    888 8888888888 8888888888 #
#  888   d88P  888  Y88b. .d88P Y88b  d88P   Y88b  d88P Y88b  d88P       888        888  #
#  8888888P" 8888888 "Y88888P"   "Y8888P"     "Y8888P"   "Y8888P"        888        888  #
#                                                                                        # 
##########################################################################################
#
# Wrangle Synthetic Medical Record Data -- Part 2
#
##########################################################################################

## Fetch data
(This is the same data used in the previous homework--no need to re-fetch it)
Go here: https://drive.google.com/drive/folders/0B5Wu8z15TIbiLUhvcWxaMDJCRGs

Get this data set: 10000-Patients.zip

unzip it into the Data folder

In [2]:
import pandas as pd
import pylab as plt
import numpy as np
import datetime as dt

In [3]:
##  ###################################################################################################################
#
# You do:  
#    Ensure you have SQLite3 available to your system.  If not, install it via pip, conda, etc.
import sqlite3

In [4]:
# Read data in to data frames

p = pd.read_csv('../Data/10000-Patients/PatientCorePopulatedTable.txt', header=0,sep = '\t')
a = pd.read_csv('../Data/10000-Patients/AdmissionsCorePopulatedTable.txt', header=0, sep='\t')
d = pd.read_csv('../Data/10000-Patients/AdmissionsDiagnosesCorePopulatedTable.txt', header=0, sep='\t')  
l = pd.read_csv('../Data/10000-Patients/LabsCorePopulatedTable.txt', header=0, sep='\t')            


In [5]:
#p.loc[p.PatientID=='915BC24E-8C44-4D33-A386-CEA965B83F32']

In [6]:
# Convert date columns to pandas date objects
# Show the type
type(p.PatientDateOfBirth[3])

for table in [p,a,d,l]:
    for col in table.columns:
        if 'date' in col.lower():
            print('Converting {}'.format(col))
            table[col] = pd.to_datetime(table[col])

Converting PatientDateOfBirth
Converting AdmissionStartDate
Converting AdmissionEndDate
Converting LabDateTime


 SQLite3 is a "lightweight" database management system.  Within Python you create a "connection" to the dabaase 
 (here we called it "conn") and you can then send SQL queries to the connection, getting standard data frames in 
 return.



In [7]:

conn = sqlite3.connect(':memory:')  # this makes an "in memory" database that isn't written to a disk.  You 
                                    # can replace this with a file name if you want to store the data for later
                                    # use.

print('Loading data into in-memory db--this could take a minute or two')
l.to_sql('L', conn, if_exists='replace')
d.to_sql('D', conn, if_exists='replace')
a.to_sql('A', conn, if_exists='replace')
p.to_sql('P', conn, if_exists='replace')

Loading data into in-memory db--this could take a minute or two


## Basic SQL queries


In [8]:
# To query via SQLite3, write a query as a Python string and "execute it" read_sql_query() method.
#
# Note:  If you are accessing data already in a different database managemnt system (MySQL/Access/etc) there are
#  interfaces available which give you a "conn" to that data as well.  You're not limited to SQLite with this
#  method.


sql = """
SELECT * from L
LIMIT 10
"""

df2 = pd.read_sql_query(sql, conn)
df2



,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,0,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: HEMATOCRIT,40.7,%,1946-09-07 22:20:26.677000
1,1,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: ANION GAP,8.4,mmol/L,1946-09-07 11:52:58.600000
2,2,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: LYMPHOCYTES,4.7,k/cumm,1946-09-07 06:08:57.303000
3,3,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: HEMOGLOBIN,15.9,gm/dl,1946-09-07 19:16:10.057000
4,4,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: SODIUM,146.6,mmol/L,1946-09-07 14:03:11.003000
5,5,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: ALBUMIN,3.3,gm/dL,1946-09-08 04:06:53.967000
6,6,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: BUN,17.1,mg/dL,1946-09-07 19:41:39.247000
7,7,915BC24E-8C44-4D33-A386-CEA965B83F32,1,CBC: NEUTROPHILS,8.4,k/cumm,1946-09-07 05:33:34.320000
8,8,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: CALCIUM,8.7,mg/dL,1946-09-07 16:23:41.213000
9,9,915BC24E-8C44-4D33-A386-CEA965B83F32,1,METABOLIC: GLUCOSE,110.5,mg/dL,1946-09-07 09:40:39.467000


In [9]:

# Use the WHERE statement

# Example: Query all labs from admission-id #3

sql = """
SELECT *
FROM L
WHERE AdmissionID==3;
"""
pd.read_sql_query(sql, conn)
#df2


,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,163,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: MCHC,31.6,g/dl,2008-10-08 10:11:47.543000
1,164,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: MCH,29.6,pg,2008-10-08 17:55:31.247000
2,165,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: CREATININE,0.6,mg/dL,2008-10-08 15:36:59.533000
3,166,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: WHITE BLOOD CELL COUNT,11.4,k/cumm,2008-10-08 09:31:41.360000
4,167,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: BUN,14.5,mg/dL,2008-10-08 07:56:07.697000
5,168,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: RDW,11.1,%,2008-10-08 03:24:55.450000
6,169,915BC24E-8C44-4D33-A386-CEA965B83F32,3,URINALYSIS: RED BLOOD CELLS,0.3,rbc/hpf,2008-10-08 23:39:43.667000
7,170,915BC24E-8C44-4D33-A386-CEA965B83F32,3,CBC: ABSOLUTE LYMPHOCYTES,23.1,%,2008-10-08 18:41:18.467000
8,171,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: CARBON DIOXIDE,20.9,mmol/L,2008-10-08 04:50:40.677000
9,172,915BC24E-8C44-4D33-A386-CEA965B83F32,3,METABOLIC: GLUCOSE,93.1,mg/dL,2008-10-08 16:40:24.897000


In [10]:
# Example: 
# Query just the time and value for CBC: MCHC labs from admission-id #3, ensuring they are sorted in time order


sql = """
SELECT LabDateTime AS Time, LabValue as Value
FROM L
WHERE AdmissionID=3 AND LabName LIKE "%MCHC"
ORDER BY Time
"""

df3 = pd.read_sql_query(sql, conn)
df3.head()

,Time,Value
0,1948-10-17 08:30:55.433000,35.8
1,1948-10-18 23:34:59.213000,39.9
2,1948-10-19 09:20:36.913000,31.1
3,1948-10-21 22:42:59.537000,33.0
4,1948-10-23 05:08:27.097000,31.4


In [11]:
# You would like to explore the relationship between lab-orders and depression
#
###################################################################################################################
#
# You do:
#  1) 
#   Generate a list of all (unique) diagnosis codes with the term "Major depressive disorder" in the description
#
#  2) Using a JOIN statement, join the Diagosis and Labs tables on PatientID and print the first 40 results

In [12]:
# 1)
    mdd_query = """
    SELECT DISTINCT(PrimaryDiagnosisCode) PDC
    FROM D
    WHERE INSTR(LOWER(PrimaryDiagnosisDescription), "major depressive disorder")
    """
    df4 = pd.read_sql_query(mdd_query,conn)
    df4.head()

## Instructor comment
##  I'm not familiar with "INSTR"  (I'll look it up later).   It seems to have missed some, though,
##  as I have 15 unique codes.

,PDC
0,F32.4
1,F33
2,F33.4
3,F32.1
4,F32.5


In [15]:
# 2) 
join_query = """
SELECT *
FROM L
JOIN D on D.PatientID = L.PatientID
WHERE INSTR(LOWER(D.PrimaryDiagnosisDescription),"major depressive disorder")
"""
df5 = pd.read_sql_query(join_query,conn)
df5.head(40)

,index,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime,index,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,61484,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,CBC: MEAN CORPUSCULAR VOLUME,91.9,fl,1963-12-27 17:39:50.400000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
1,61485,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,METABOLIC: CARBON DIOXIDE,22.4,mmol/L,1963-12-27 22:31:08.147000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
2,61486,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,METABOLIC: BILI TOTAL,0.4,mg/dL,1963-12-28 01:05:28.767000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
3,61487,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,CBC: EOSINOPHILS,0.2,k/cumm,1963-12-28 05:46:29.857000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
4,61488,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,CBC: PLATELET COUNT,332.7,k/cumm,1963-12-27 15:25:47.783000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
5,61489,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,URINALYSIS: SPECIFIC GRAVITY,1.0,no unit,1963-12-28 01:26:09.660000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
6,61490,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,METABOLIC: CALCIUM,9.9,mg/dL,1963-12-28 06:28:16.390000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
7,61491,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,CBC: ABSOLUTE LYMPHOCYTES,20.0,%,1963-12-28 01:28:27.357000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
8,61492,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,CBC: RDW,9.9,%,1963-12-27 21:52:01.270000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."
9,61493,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,CBC: MONOCYTES,0.5,k/cumm,1963-12-27 12:52:23.333000,33,EEB94FF2-5E61-4B22-AB77-4F53B8932178,1,F32.4,"Major depressive disorder, single episode, in ..."


In [19]:

###################################################################################################################
#
# You do:
#  1) Create a list of each lab name ordered and it's count for any patient who also had a diagnosis which 
#     includes the term "Major depressive disorder"

join_query = """
SELECT L.PatientID, LabName, COUNT(*) as NumOrders
FROM L
JOIN D on D.PatientID = L.PatientID
WHERE INSTR(LOWER(D.PrimaryDiagnosisDescription),"major depressive disorder")
GROUP BY L.PatientID,LabName"""
df6 = pd.read_sql_query(join_query,conn)

## Again, INSTR seems to be giving you quite different counts.   See my solution for details.

In [21]:
df6.set_index(["PatientID","LabName"])

NumOrders
PatientID                            LabName                                 
0056BC12-2FD9-4B7E-9599-1F3A0636BB81 CBC: ABSOLUTE LYMPHOCYTES             31
                                     CBC: ABSOLUTE NEUTROPHILS             30
                                     CBC: BASOPHILS                        29
                                     CBC: EOSINOPHILS                      32
                                     CBC: HEMATOCRIT                       25
                                     CBC: HEMOGLOBIN                       31
                                     CBC: LYMPHOCYTES                      31
                                     CBC: MCH                              30
                                     CBC: MCHC                             28
                                     CBC: MEAN CORPUSCULAR VOLUME          30
                                     CBC: MONOCYTES                        30
                                     CBC: NEUTROPHILS                      34
                                     CBC: PLATELET COUNT                   30
                                     CBC: RDW                              29
                                     CBC: RED BLOOD CELL COUNT             32
                                     CBC: WHITE BLOOD CELL COUNT           30
                                     METABOLIC: ALBUMIN                    36
                                     METABOLIC: ALK PHOS                   30
                                     METABOLIC: ALT/SGPT                   28
                                     METABOLIC: ANION GAP                  32
                                     METABOLIC: AST/SGOT                   33
                                     METABOLIC: BILI TOTAL                 35
                                     METABOLIC: BUN                        30
                                     METABOLIC: CALCIUM                    25
                                     METABOLIC: CARBON DIOXIDE             30
                                     METABOLIC: CHLORIDE                   31
                                     METABOLIC: CREATININE                 33
                                     METABOLIC: GLUCOSE                    32
                                     METABOLIC: POTASSIUM                  30
                                     METABOLIC: SODIUM                     31
...                                                                       ...
FF7D0FAC-18C4-4D0B-8FDD-91152493F9C6 CBC: HEMOGLOBIN                       38
                                     CBC: LYMPHOCYTES                      37
                                     CBC: MCH                              36
                                     CBC: MCHC                             36
                                     CBC: MEAN CORPUSCULAR VOLUME          37
                                     CBC: MONOCYTES                        36
                                     CBC: NEUTROPHILS                      36
                                     CBC: PLATELET COUNT                   29
                                     CBC: RDW                              34
                                     CBC: RED BLOOD CELL COUNT             38
                                     CBC: WHITE BLOOD CELL COUNT           42
                                     METABOLIC: ALBUMIN                    35
                                     METABOLIC: ALK PHOS                   32
                                     METABOLIC: ALT/SGPT                   33
                                     METABOLIC: ANION GAP                  35
                                     METABOLIC: AST/SGOT                   32
                                     METABOLIC: BILI TOTAL                 29
                                     METABOLIC: BUN                        38
                                     METABOLIC: CALCIUM                    35
            